# Second level GLM using Nipype FSL

Notebook compiled by Yibei Chen

In this notebook, we recreate the whole preprocess of FSL GUI using nipype code. For each nipype node, we list the corresponding fsl command from the log file. The dataset we use is a Flanker task, which can be downloaded [here](https://openneuro.org/datasets/ds000102/versions/00001).

We also borrow some helps from this [document](https://nipype.readthedocs.io/en/latest/users/examples/fmri_fsl.html). 

## Preparation
Import all the relevant libraries needed for the preprocessing stage.

In [1]:
from __future__ import print_function
from __future__ import division
from builtins import str
from builtins import range

import os, stat  # system functions
import getpass
from glob import glob

import numpy as np
from nipype import Function
import nipype.interfaces.io as nio  # Data i/o
import nipype.interfaces.fsl as fsl  # fsl
import nipype.interfaces.utility as util  # utility
import nipype.pipeline.engine as pe  # pypeline engine
import nipype.algorithms.modelgen as model  # model generation
import nipype.algorithms.rapidart as ra  # artifact detection

%autosave 5

Autosaving every 5 seconds


Set up data path

In [2]:
# Get current user
user = getpass.getuser()
print('Running code as: ', user)

# Input: Set the first-level result path
data_dir = '/home/{}/out/fsl/hw2/level1_results/'.format(user)
# Output: Set path where nipype will store stepwise results
exp_dir = '/home/{}/out/fsl/hw2/'.format(user)

try:
    os.mkdir(exp_dir)
except OSError as error:
    print(error)
    
    
# Grant root write access to our output files 
os.chmod(exp_dir, os.stat(exp_dir).st_mode | ((stat.S_IRWXU | stat.S_IRWXO)))

Running code as:  yc
[Errno 17] File exists: '/home/yc/out/fsl/hw2/'


Start the workflow

In [3]:
wf = pe.Workflow(name='level2', base_dir=exp_dir)
wf.config["execution"]["crashfile_format"] = "txt"

The following two nodes (`infosource` & `dg`) together define all inputs required for the preprocessing workflow

In [4]:
# we want to group the outcome by contrast not subject
contr_list = [1,2,3,4]
infosource = pe.Node(util.IdentityInterface(fields=["contr_id"]),
                  name="infosource")
infosource.iterables = [("contr_id", contr_list)]

In [5]:
# here we use SelectFiles, instead of DataGrabber, because the former is more flexible with formatting syntax
templates = {
    "reg_copes":"*/reg_copes/*/*/cope{contr_id}_flirt.nii.gz",
    "reg_varcopes":"*/reg_varcopes/*/*/varcope{contr_id}_flirt.nii.gz",
    "reg_masks":"*/reg_masks/*/*/*.nii.gz"
}
dg = pe.Node(interface=nio.SelectFiles(templates),
             name="selectfiles")
dg.inputs.base_directory = data_dir

wf.connect([
        (infosource, dg, [("contr_id", "contr_id")])
])

## Second-level GLM

Combining results from multiple runs of one subject into one

### Higher-level input files preparation

#### Step 1: Merge registered copes & varcopes & masks


**Corresponding FSL command:**
```
/usr/local/fsl/bin/fslmerge -t mask (masks from all 52 inputs)
/usr/local/fsl/bin/fslmerge -t cope (copes from all 52 inputs)
/usr/local/fsl/bin/fslmerge -t varcop (varcopes from all 52 inputs)
```

In [6]:
copemerge = pe.Node(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="copemerge")

varcopemerge = pe.Node(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="varcopemerge")

maskmerge = pe.Node(
    interface=fsl.Merge(dimension='t'),
    iterfield=['in_files'],
    name="maskmerge")


def sort_copes(files):
    numelements = len(files[0])
    outfiles = []
    for i in range(numelements):
        outfiles.insert(i, [])
        for j, elements in enumerate(files):
            outfiles[i].append(elements[i])
    return outfiles



wf.connect(dg, 'reg_copes', copemerge, 'in_files')
wf.connect(dg, 'reg_varcopes', varcopemerge, 'in_files')
wf.connect(dg, 'reg_masks', maskmerge, 'in_files')


#### Step 2: Making mask

In FSL, there are many commands about `maskunique`, which is unless for the second level. We can ignore it.

**Corresponding FSL command:**
```
/usr/local/fsl/bin/fslmaths mask -Tmin mask
```

In [7]:
# /usr/local/fsl/bin/fslmaths mask -Tmin mask
minmask = pe.Node(
    interface=fsl.ImageMaths(op_string='-Tmin'),
    iterfield=['in_file'],
    name='minmask')

wf.connect(maskmerge, 'merged_file', minmask, 'in_file')

#### Step 3: Masking copes & varcopes

**Corresponding FSL command:**

we have four contrasts so the following commands repeat four times

```
/usr/local/fsl/bin/fslmaths cope1 -mas mask cope1
/usr/local/fsl/bin/fslmaths varcope1 -mas mask varcope1
```

In [8]:
maskcope = pe.Node(
    interface=fsl.ImageMaths(op_string='-mas'),
    iterfield=['in_file', 'in_file2'],
    name='maskcope')

maskvarcope = pe.Node(
    interface=fsl.ImageMaths(op_string='-mas'),
    iterfield=['in_file', 'in_file2'],
    name='maskvarcope')

wf.connect(copemerge, 'merged_file', maskcope, 'in_file')
wf.connect(minmask, 'out_file', maskcope, 'in_file2')
wf.connect(varcopemerge, 'merged_file', maskvarcope, 'in_file')
wf.connect(minmask, 'out_file', maskvarcope, 'in_file2')

### Set up second-level contrasts and fixed-effects

In [11]:
def get_contrasts_l2(in_files):
    import numpy as np
    total = len(in_files)
    print(in_files)
    print(total)
    n_sub = 26
    ev_list = ['ev'+str(x) for x in range(1,n_sub+1)]
    weight_mtx = np.zeros((26,26))
    weight_mtx = weight_mtx.astype(np.float64)
    np.fill_diagonal(weight_mtx,1.)
    contr = ['','T',ev_list,list(weight_mtx[0])]
    contr_lst = np.tile(contr, (n_sub, 1))
    contr_lst = [list(x) for x in contr_lst]
    for i in range(n_sub):
        contr_lst[i][3] = list(weight_mtx[i])

    reg_dict = {k:None for k in ev_list}
    for k in reg_dict.keys():
        start_lst = [0.0] * total
        idx = ev_list.index(k)
        start_idx = idx*2
        end_idx = idx*2 + 1
        start_lst[start_idx] = 1.
        start_lst[end_idx] = 1.
        reg_dict[k] = start_lst
        
    return contr_lst, reg_dict

contrastgen_l2 = pe.Node(util.Function(input_names=['in_files'],
                                    output_names=['contr_lst', 'reg_dict'],
                                    function=get_contrasts_l2),
                      iterfield=['in_files'],
                      name='contrastgen_l2')

wf.connect(dg,'reg_copes', contrastgen_l2, 'in_files')

Nipype recommands using [L2Model](https://nipype.readthedocs.io/en/latest/api/generated/nipype.interfaces.fsl.model.html#l2model), which only works for the single subject. It takes the number of runs (copes at the first level) as input and does estimations for subject one by one.
Instead, we use [MultipleRegressDesign](https://nipype.readthedocs.io/en/latest/api/generated/nipype.interfaces.fsl.model.html#multipleregressdesign). As it's name indicates, this one can deal with multiple predictors (subjects) at the same time.

In [12]:
level2model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l2model')

wf.connect([(contrastgen_l2, level2model, [('contr_lst','contrasts'),
                                        ('reg_dict','regressors')])])

In [13]:
level2estimate = pe.Node(
    interface=fsl.FLAMEO(run_mode='fe'),
    name="level2estimate",
    iterfield=['cope_file', 'var_cope_file'])

wf.connect([
    (maskcope, level2estimate, [('out_file', 'cope_file')]),
    (maskvarcope, level2estimate, [('out_file', 'var_cope_file')]),
    (minmask, level2estimate, [('out_file', 'mask_file')]),
    (level2model, level2estimate, [('design_mat', 'design_file'),
                                   ('design_con', 't_con_file'), 
                                   ('design_grp', 'cov_split_file')]),
])

In [14]:
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=os.path.join(exp_dir, "level2_results")

int2string = lambda x: 'contrast_'+str(x)
    
wf.connect(infosource, ('contr_id',int2string), datasink, 'container')
wf.connect([(level2estimate, datasink, [('stats_dir', 'stats_dir')])])

In [16]:
# Run Workflow
wf.run(plugin="MultiProc", plugin_args={"n_procs": 8})

211208-05:42:38,303 nipype.workflow INFO:
	 Workflow level2 settings: ['check', 'execution', 'logging', 'monitoring']
211208-05:42:38,349 nipype.workflow INFO:
	 Running in parallel.
211208-05:42:38,352 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 27.96/27.96, Free processors: 8/8.
211208-05:42:38,426 nipype.workflow INFO:
	 [Node] Setting-up "level2.selectfiles" in "/home/yc/out/fsl/hw2/level2/_contr_id_4/selectfiles".
211208-05:42:38,429 nipype.workflow INFO:
	 [Node] Setting-up "level2.selectfiles" in "/home/yc/out/fsl/hw2/level2/_contr_id_3/selectfiles".
211208-05:42:38,431 nipype.workflow INFO:
	 [Node] Setting-up "level2.selectfiles" in "/home/yc/out/fsl/hw2/level2/_contr_id_2/selectfiles".
211208-05:42:38,434 nipype.workflow INFO:
	 [Node] Setting-up "level2.selectfiles" in "/home/yc/out/fsl/hw2/level2/_contr_id_1/selectfiles".
211208-05:42:38,481 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")